In [24]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

import seaborn as sns

### Training model without Feature Construction & Feature Splitting

In [3]:
df = pd.read_csv('titanic.csv')[['Age','Pclass','SibSp','Parch','Survived']]

In [4]:
df.head()

,Age,Pclass,SibSp,Parch,Survived
0,22.0,3,1,0,0
1,38.0,1,1,0,1
2,26.0,3,0,0,1
3,35.0,1,1,0,1
4,35.0,3,0,0,0


In [5]:
df.shape

(891, 5)

In [6]:
df.isnull().sum()

Age         177
Pclass        0
SibSp         0
Parch         0
Survived      0
dtype: int64

In [7]:
df.dropna(inplace=True)

In [8]:
df.isnull().sum()

Age         0
Pclass      0
SibSp       0
Parch       0
Survived    0
dtype: int64

In [9]:
df.shape

(714, 5)

In [11]:
df.sample(5)

,Age,Pclass,SibSp,Parch,Survived
765,51.0,1,1,0,1
3,35.0,1,1,0,1
412,33.0,1,1,0,1
307,17.0,1,1,0,1
788,1.0,3,1,2,1


In [13]:
X = df.drop('Survived', axis=1)
X.head()

,Age,Pclass,SibSp,Parch
0,22.0,3,1,0
1,38.0,1,1,0
2,26.0,3,0,0
3,35.0,1,1,0
4,35.0,3,0,0


In [15]:
y = df['Survived']
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [18]:
lr = LogisticRegression()

In [19]:
lr.fit(X_train, y_train)

LogisticRegression()

In [23]:
y_test

149    0
407    1
53     1
369    1
818    0
      ..
819    0
164    0
363    0
56     1
136    1
Name: Survived, Length: 143, dtype: int64

In [22]:
y_predict = lr.predict(X_test)

y_predict

array([0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1], dtype=int64)

In [25]:
accuracy_score(y_test, y_predict)

0.6923076923076923

In [26]:
np.mean(cross_val_score(lr,X,y,scoring='accuracy',cv=20))

0.6933333333333332

### Applying Feature Construction

In [28]:
X['Family_size'] = X['SibSp'] + X['Parch'] + 1 # Combining Siblings and Parch column to create new column named family size

In [29]:
X.head()

,Age,Pclass,SibSp,Parch,Family_size
0,22.0,3,1,0,2
1,38.0,1,1,0,2
2,26.0,3,0,0,1
3,35.0,1,1,0,2
4,35.0,3,0,0,1


In [ ]:
# Now we can drop the columns SibSp and Parch



In [31]:
def fam_grouping(num):
    if num == 1:
        # travelling alone
        return 0
    elif num > 1 and num <= 4:
        # small family
        return 1
    else:
        # large family
        return 2

In [32]:
fam_grouping(3)

1

In [35]:
X['Family_type'] = X['Family_size'].apply(fam_grouping)

In [36]:
X.sample(5)

,Age,Pclass,SibSp,Parch,Family_size,Family_type
770,24.0,3,0,0,0,2
871,47.0,1,1,1,1,0
715,19.0,3,0,0,0,2
151,22.0,1,1,0,1,0
626,57.0,2,0,0,0,2


In [37]:
X.drop(['SibSp', 'Parch', 'Family_size'], axis=1, inplace=True)

In [38]:
X.head()

,Age,Pclass,Family_type
0,22.0,3,0
1,38.0,1,0
2,26.0,3,2
3,35.0,1,0
4,35.0,3,2


In [39]:
np.mean(cross_val_score(lr,X,y,scoring='accuracy',cv=20))

# accuracy Increased

0.7312698412698414

### Feature Splitting

In [40]:
df = pd.read_csv('titanic.csv')

In [41]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [42]:
df['Name']

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 891, dtype: object

In [47]:
# df['Title'] = df['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]

df['Title'] = df['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]

In [49]:
df[['Name', 'Title']]

,Name,Title
0,"Braund, Mr. Owen Harris",Mr
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Mrs
2,"Heikkinen, Miss. Laina",Miss
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Mrs
4,"Allen, Mr. William Henry",Mr
...,...,...
886,"Montvila, Rev. Juozas",Rev
887,"Graham, Miss. Margaret Edith",Miss
888,"Johnston, Miss. Catherine Helen ""Carrie""",Miss
889,"Behr, Mr. Karl Howell",Mr


In [53]:
(df.groupby('Title').mean()['Survived']).sort_values(ascending=False)

Title
the Countess    1.000000
Mlle            1.000000
Sir             1.000000
Ms              1.000000
Lady            1.000000
Mme             1.000000
Mrs             0.792000
Miss            0.697802
Master          0.575000
Col             0.500000
Major           0.500000
Dr              0.428571
Mr              0.156673
Jonkheer        0.000000
Rev             0.000000
Don             0.000000
Capt            0.000000
Name: Survived, dtype: float64

In [56]:
df['Title'].value_counts()

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Mlle              2
Major             2
Col               2
the Countess      1
Capt              1
Ms                1
Sir               1
Lady              1
Mme               1
Don               1
Jonkheer          1
Name: Title, dtype: int64

In [57]:
df['Is_Married'] = 0
df['Is_Married'].loc[df['Title'] == 'Mrs'] = 1

C:\Users\User\AppData\Local\Temp\ipykernel_668\2254989826.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Is_Married'].loc[df['Title'] == 'Mrs'] = 1


In [59]:
df['Is_Married'].head()

0    0
1    1
2    0
3    1
4    0
Name: Is_Married, dtype: int64